In [49]:
import os, glob
import numpy as np
import pandas as pd
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, matthews_corrcoef,
    roc_curve
)

# ---- Folders and model types to evaluate ----
model_folders = {
    "RF (Proteomics + Demographics)": "/Users/adithyamadduri/Desktop/Projects/proteomics_RF(ANML+Meta)",
    "LGBM (Proteomics + Demographics)": "/Users/adithyamadduri/Desktop/Projects/proteomics_LGBM(ANML+Meta)",
    "LGBM (Protein Ratios)": "/Users/adithyamadduri/Desktop/Projects/proteomics_LGBM(ANML+Meta)_RATIOS_fixed_rfe",
    "LGBM (Protein Ratios + Demographics)": "/Users/adithyamadduri/Desktop/Projects/proteomics_LGBM(ANML+Meta)_RATIOS_Demo",
    "RF (Genes + Demographics)": "/Users/adithyamadduri/Desktop/Projects/ml4h_project/RF(Genes+Demo)",
    "LGBM (Genes + Demographics)": "/Users/adithyamadduri/Desktop/Projects/ml4h_project/LGBM(Genes+Demo)",
    "LGBM (Gene Ratios)": "/Users/adithyamadduri/Desktop/Projects/Genomics_LGBM(Genomics+Meta)_RATIOS_fixed_rfe",
    "LGBM (Gene Ratios + Demographics)": "/Users/adithyamadduri/Desktop/Projects/Genomics_LGBM(Genomics+Meta)_RATIOS_Demo"
}


# ---- Metrics to compute ----
def compute_metrics(y_true, y_score, y_pred):
    return {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, zero_division=0),
        "Recall": recall_score(y_true, y_pred, zero_division=0),
        "F1": f1_score(y_true, y_pred, zero_division=0),
        "ROC AUC": roc_auc_score(y_true, y_score),
        "PR AUC": average_precision_score(y_true, y_score),
        "MCC": matthews_corrcoef(y_true, y_pred),
    }

# ---- Find Youden's J optimal threshold ----
def optimal_threshold_youden(y_true, y_score):
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    youden_j = tpr - fpr
    best_idx = np.argmax(youden_j)
    return thresholds[best_idx]

# ---- Aggregate per-model results ----
rows = []

for model_name, folder in model_folders.items():
    files = glob.glob(os.path.join(folder, "seed*_*.csv"))
    if not files:
        print(f"No files found for {model_name}. Skipping.")
        continue

    # Extract class names
    def extract_class(p): return "_".join(os.path.basename(p).split("_")[1:]).replace(".csv", "")
    class_names = sorted(set(extract_class(f) for f in files))

    for cls in class_names:
        cls_files = sorted(glob.glob(os.path.join(folder, f"seed*_{cls}.csv")))

        metric_lists = {k: [] for k in ["Accuracy", "Precision", "Recall", "F1", "ROC AUC", "PR AUC", "MCC"]}

        for f in cls_files:
            df = pd.read_csv(f)
            y_true = df["y_true"].astype(int).values
            y_score = df["y_score"].astype(float).values

            try:
                # Compute optimal threshold for Youden's J
                opt_thresh = optimal_threshold_youden(y_true, y_score)
                y_pred = (y_score >= opt_thresh).astype(int)

                metrics = compute_metrics(y_true, y_score, y_pred)
                for k, v in metrics.items():
                    metric_lists[k].append(v)
            except Exception as e:
                print(f"Error in {f}: {e}")

        # Aggregate mean ± std
        summary = {
            "Model": model_name,
            "Class": cls
        }
        for k in metric_lists:
            vals = np.array(metric_lists[k])
            summary[f"{k} (mean±std)"] = f"{np.mean(vals):.3f} ± {np.std(vals):.3f}"
        rows.append(summary)

# ---- Final DataFrame ----
summary_df = pd.DataFrame(rows)
summary_df = summary_df.sort_values(by=["Model", "Class"])
summary_df.reset_index(drop=True, inplace=True)

# ---- Save ----
summary_df.to_csv("ML4H_model_eval_summary.csv", index=False)
print(summary_df)

                                   Model               Class  \
0      LGBM (Gene Ratios + Demographics)     Healthy_Control   
1      LGBM (Gene Ratios + Demographics)           Resilient   
2      LGBM (Gene Ratios + Demographics)  Symptomatic_Non-AD   
3      LGBM (Gene Ratios + Demographics)          Typical_AD   
4                     LGBM (Gene Ratios)     Healthy_Control   
5                     LGBM (Gene Ratios)           Resilient   
6                     LGBM (Gene Ratios)  Symptomatic_Non-AD   
7                     LGBM (Gene Ratios)          Typical_AD   
8            LGBM (Genes + Demographics)     Healthy_Control   
9            LGBM (Genes + Demographics)           Resilient   
10           LGBM (Genes + Demographics)  Symptomatic_Non-AD   
11           LGBM (Genes + Demographics)          Typical_AD   
12  LGBM (Protein Ratios + Demographics)                  AD   
13  LGBM (Protein Ratios + Demographics)              ADplus   
14  LGBM (Protein Ratios + Demographics)

In [59]:
import pandas as pd

# Load your CSV
df = pd.read_csv("ML4H_model_eval_summary.csv")

# Pick the columns you want in the LaTeX table
# (adjust order/columns if needed)
cols = [
    "Model",
    "Class",
    "Accuracy (mean±std)",
    "Precision (mean±std)",
    "Recall (mean±std)",
    "F1 (mean±std)",
    "ROC AUC (mean±std)",
    "PR AUC (mean±std)",
    "MCC (mean±std)"
]

df = df[cols]

# Escape underscores for LaTeX
df = df.replace("_", r"\_", regex=True)

# Build LaTeX table body
rows = []
for _, row in df.iterrows():
    row_str = " & ".join(str(row[c]) for c in cols) + r" \\"
    rows.append(row_str)

# Wrap into full table environment
latex = r"""
\begin{table*}[htbp]
\floatconts
  {tab:full_eval}%
  {\caption{Per-model, per-class evaluation (mean $\pm$ std across 5 seeds).}}%
  {%
  \scriptsize
    \begin{tabular}{lllllllll}
    \toprule
    Model & Class & Accuracy & Precision & Recall & F1 & ROC AUC & PR AUC & MCC \\
    \midrule
""" + "\n".join(rows) + r"""
    \bottomrule
    \end{tabular}
    \par
  }
\end{table*}
"""

# Save or print
with open("ML4H_model_eval_summary_table.tex", "w") as f:
    f.write(latex)

print(latex)


\begin{table*}[htbp]
\floatconts
  {tab:full_eval}%
  {\caption{Per-model, per-class evaluation (mean $\pm$ std across 5 seeds).}}%
  {%
  \scriptsize
    \begin{tabular}{lllllllll}
    \toprule
    Model & Class & Accuracy & Precision & Recall & F1 & ROC AUC & PR AUC & MCC \\
    \midrule
LGBM (Gene Ratios + Demographics) & Healthy\_Control & 0.800 ± 0.020 & 0.856 ± 0.046 & 0.655 ± 0.078 & 0.737 ± 0.040 & 0.829 ± 0.018 & 0.844 ± 0.022 & 0.599 ± 0.040 \\
LGBM (Gene Ratios + Demographics) & Resilient & 0.658 ± 0.075 & 0.375 ± 0.048 & 0.711 ± 0.127 & 0.481 ± 0.022 & 0.704 ± 0.034 & 0.408 ± 0.036 & 0.308 ± 0.032 \\
LGBM (Gene Ratios + Demographics) & Symptomatic\_Non-AD & 0.612 ± 0.143 & 0.126 ± 0.030 & 0.682 ± 0.160 & 0.206 ± 0.032 & 0.630 ± 0.048 & 0.161 ± 0.045 & 0.161 ± 0.039 \\
LGBM (Gene Ratios + Demographics) & Typical\_AD & 0.682 ± 0.006 & 0.457 ± 0.006 & 0.851 ± 0.041 & 0.595 ± 0.014 & 0.770 ± 0.022 & 0.493 ± 0.040 & 0.419 ± 0.026 \\
LGBM (Gene Ratios) & Healthy\_Control & 0.593

In [31]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, auc

# --- Folders ---
baseline_folder = "/Users/adithyamadduri/Desktop/Projects/proteomics_LGBM(ANML+Meta)"                 # LGBM (Proteomics)
ratios_folder   = "/Users/adithyamadduri/Desktop/Projects/proteomics_LGBM(ANML+Meta)_RATIOS_fixed_rfe"  # LGBM (Protein Ratios)

classes = ["MCI", "NCI", "AD", "AD+"]
seeds   = [1, 2, 3, 4, 5]

def safe_cls(c):
    return c.replace("+","plus").replace(" ","_").replace("/","-")

def mean_auc_like_plot(folder):
    """Compute per-class mean AUC exactly like the plotting code:
       for each seed file: fpr,tpr = roc_curve(...); mean auc = mean(auc(fpr,tpr))."""
    mean_by_class = {}
    used_seeds = {}
    for cls in classes:
        aucs, used = [], []
        for seed in seeds:
            path = os.path.join(folder, f"seed{seed}_{safe_cls(cls)}.csv")
            if not os.path.exists(path):
                continue
            df = pd.read_csv(path)
            y_true  = df["y_true"].astype(int).values
            y_score = df["y_score"].astype(float).values
            fpr, tpr, _ = roc_curve(y_true, y_score)  # EXACTLY as in your plot
            aucs.append(auc(fpr, tpr))                # no FPR dedup, no roc_auc_score
            used.append(seed)
        if aucs:
            mean_by_class[cls] = float(np.mean(aucs))
            used_seeds[cls] = used
    return mean_by_class, used_seeds

# Compute means for each folder
base_mean, base_used = mean_auc_like_plot(baseline_folder)
ratio_mean, ratio_used = mean_auc_like_plot(ratios_folder)

# Build table and differences (unpaired means)
common_classes = sorted(set(base_mean) & set(ratio_mean))
rows = []
for cls in common_classes:
    rows.append({
        "Class": cls,
        "base_mean_auc": base_mean[cls],
        "ratio_mean_auc": ratio_mean[cls],
        "diff": ratio_mean[cls] - base_mean[cls],
        "base_seeds": base_used.get(cls, []),
        "ratio_seeds": ratio_used.get(cls, []),
    })

df = pd.DataFrame(rows).sort_values("Class")

# Pretty print (rounding only for display)
fmt = {"base_mean_auc": lambda x: f"{x:.3f}",
       "ratio_mean_auc": lambda x: f"{x:.3f}",
       "diff": lambda x: f"{x:.3f}"}
print(df[["Class","base_mean_auc","ratio_mean_auc","diff"]].to_string(index=False, formatters=fmt))

overall = df["diff"].mean()
print(f"\nAverage ROC AUC difference across classes: {overall:.3f}")

Class base_mean_auc ratio_mean_auc  diff
   AD         0.716          0.800 0.084
  AD+         0.629          0.822 0.193
  MCI         0.518          0.651 0.133
  NCI         0.675          0.728 0.053

Average ROC AUC difference across classes: 0.116


In [33]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, auc

# --- Folders (Genomics) ---
baseline_folder = "/Users/adithyamadduri/Desktop/Projects/ml4h_project/LGBM(Genes+Demo)"                 # LGBM (Genomics + Demographics)
ratios_folder   = "/Users/adithyamadduri/Desktop/Projects/Genomics_LGBM(Genomics+Meta)_RATIOS_Demo"  # LGBM (Gene Ratios, no demo)

# --- Classes (Genomics labels) ---
classes = ["Healthy Control", "Resilient", "Symptomatic Non-AD", "Typical AD"]
seeds   = [1, 2, 3, 4, 5]

def safe_cls(c):
    return c.replace("+","plus").replace(" ","_").replace("/","-")

def mean_auc_like_plot(folder):
    """Compute per-class mean AUC exactly like the plotting code:
       for each seed file: fpr,tpr = roc_curve(...); mean auc = mean(auc(fpr,tpr))."""
    mean_by_class, used_seeds = {}, {}
    for cls in classes:
        aucs, used = [], []
        for seed in seeds:
            path = os.path.join(folder, f"seed{seed}_{safe_cls(cls)}.csv")
            if not os.path.exists(path):
                continue
            df = pd.read_csv(path)
            y_true  = df["y_true"].astype(int).values
            y_score = df["y_score"].astype(float).values
            fpr, tpr, _ = roc_curve(y_true, y_score)  # EXACTLY as in your plot
            aucs.append(auc(fpr, tpr))                # no FPR dedup, no roc_auc_score
            used.append(seed)
        if aucs:
            mean_by_class[cls] = float(np.mean(aucs))
            used_seeds[cls] = used
    return mean_by_class, used_seeds

# Compute means for each folder
base_mean, base_used = mean_auc_like_plot(baseline_folder)  # Genomics + Demo
ratio_mean, ratio_used = mean_auc_like_plot(ratios_folder)  # Gene Ratios (no demo)

# Build table and differences (unpaired means)
common_classes = sorted(set(base_mean) & set(ratio_mean))
rows = []
for cls in common_classes:
    rows.append({
        "Class": cls,
        "base_mean_auc": base_mean[cls],
        "ratio_mean_auc": ratio_mean[cls],
        "diff": ratio_mean[cls] - base_mean[cls],
        "base_seeds": base_used.get(cls, []),
        "ratio_seeds": ratio_used.get(cls, []),
    })

df = pd.DataFrame(rows).sort_values("Class")

# Pretty print (rounding only for display)
fmt = {"base_mean_auc": lambda x: f"{x:.3f}",
       "ratio_mean_auc": lambda x: f"{x:.3f}",
       "diff": lambda x: f"{x:.3f}"}
print(df[["Class","base_mean_auc","ratio_mean_auc","diff"]].to_string(index=False, formatters=fmt))

overall = df["diff"].mean() if not df.empty else float("nan")
print(f"\nAverage ROC AUC difference across classes: {overall:.3f}")

             Class base_mean_auc ratio_mean_auc  diff
   Healthy Control         0.797          0.829 0.032
         Resilient         0.645          0.704 0.059
Symptomatic Non-AD         0.530          0.630 0.100
        Typical AD         0.730          0.770 0.040

Average ROC AUC difference across classes: 0.058
